In [1]:
from datetime import datetime
import argparse
from fileinput import filename
import numpy as np
import pathlib
import argparse
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors
from matplotlib import rcParams
import torch
from torch.nn import functional as F
from matplotlib import cm
import datagen
import train
import utils
from pyevtk.hl import imageToVTK,gridToVTK
import datagen
import encoder
import decoder
import discriminator
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import IterableDataset,Dataset

[Errno 17] File exists: '/home/liyongbo/.local/share/geoist'
[Errno 17] File exists: '/home/liyongbo/.local/share/geoist/examples'
[Errno 17] File exists: '/home/liyongbo/.local/share/geoist/temp'
[Errno 17] File exists: '/home/liyongbo/.local/share/geoist/data'


In [2]:
def load_args():
    parser = argparse.ArgumentParser(description='gangravity')
    parser.add_argument('-c', '--checkpoint', default='./models/checkpoint.pt',
                        type=str, help='checkpoint file')
    parser.add_argument('-d', '--device', default='cuda:0', type=str, help='computing device')
    parser.add_argument('-l', '--gp', default=10, type=int, help='gradient penalty')
    parser.add_argument('-g', '--n_gp', default=1, type=int)
    parser.add_argument('-b', '--batch_size', default=1, type=int)
    parser.add_argument('-n', '--n_batch', default=150, type=int)
    parser.add_argument('-e', '--epochs', default=100, type=int)
    parser.add_argument('-w', '--world_size', default=3, type=int)
    parser.add_argument('--use_spectral_norm', default=False)
    args = parser.parse_args()
    return args

def load_models(rank,args,checkpoint=None):
    netDec = decoder.GravDecoder()
    netDis = discriminator.GravDiscriminator()
    netEnc = encoder.GravEncoder()
    netEnc = DDP(netEnc.to(rank),device_ids=[rank])
    #netDec = DDP(netDec.to(rank),device_ids=[rank])
    netDec = netDec.to(rank)
    netDis = DDP(netDis.to(rank),device_ids=[rank])
    if checkpoint:
        netEnc.load_state_dict(checkpoint['enc_state_dict'])
        netDis.load_state_dict(checkpoint['dis_state_dict'])
#    netEnc = torch.nn.DataParallel(netEnc)
#    netDec = torch.nn.DataParallel(netDec)
#    netDis = torch.nn.DataParallel(netDis) 
    print (netDec, netDis, netEnc)
    return (netDec, netDis, netEnc)
    
def dice(pred, target):
    smooth = 1
    num = pred.size(0)
    m1 = pred.view(num, -1)  
    m2 = target.view(num, -1)  
    intersection = m1 * m2
    loss = (2. * intersection.sum(1) + smooth) / ((m1*m1).sum(1) + (m2*m2).sum(1) + smooth)
    return loss.sum()/num
def my_loss(pre_y, tru_y): 
    loss = 1 - dice(pre_y, tru_y)
    return loss


In [3]:
if __name__ == '__main__':

    args_gen = load_args()
    checkpoint = torch.load(args_gen.checkpoint)
    saved_model = ['enc_state_dict','dis_state_dict']
    for model_key in saved_model:
        new_keys = list(map(lambda x:x[7:],checkpoint[model_key].keys()))
        checkpoint[model_key] = dict(zip(new_keys,list(checkpoint[model_key].values())))
    netDec,netDis,netEnc = train.load_models(args_gen,checkpoint,nzyx=nzyx)
    netDec.eval()
    netDis.eval()
    netEnc.eval()
    train_length = 10
    tes_loss  = 0
    tes_metrix = 0
    for index_train in  range(train_length):
        index_train = index_train*10 +1
        with torch.no_grad():

            modeltest = 'model{}.npy'.format(str(index_train))
            testfile = os.path.join(os.getcwd(), 'traindatasets', "models", modeltest)
            data = np.load(testfile)
            input_data = data[0:2, : , : ]
            oue_put = data[2, : , : ]
            
            input_data = torch.from_numpy(input_data)
            input_data = input_data.unsqueeze(0)

            density_rec = netEnc(input_data)
            rec = np.array(density_rec.sequeeze(0))
            plt.figure(figsize=(12, 4), dpi = 150)
            ax1 = plt.subplot(131)
            plt.imshow(input_data[0,:,:], origin='lower')
            ax2 = plt.subplot(132)
            plt.imshow(rec, origin='lower')
            ax2 = plt.subplot(133)
            plt.imshow(input_data[0,:,:]-rec, origin='lower')
            plt.savefig("{}.png".format(index_train))
    

usage: ipykernel_launcher.py [-h] [-c CHECKPOINT] [-d DEVICE] [-l GP]
                             [-g N_GP] [-b BATCH_SIZE] [-n N_BATCH]
                             [-e EPOCHS] [-w WORLD_SIZE]
                             [--use_spectral_norm USE_SPECTRAL_NORM]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9023 --control=9021 --hb=9020 --Session.signature_scheme="hmac-sha256" --Session.key=b"70f29743-6490-4fcc-ad02-fe7acd4f8adc" --shell=9022 --transport="tcp" --iopub=9024 --f=/home/liyongbo/.local/share/jupyter/runtime/kernel-18728X76L8RpacAHR.json


SystemExit: 2

/home/liyongbo/anaconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
